## Match Filardo data to PubMed and extract author:gender data

In [1]:
from Bio import Entrez
import pandas as pd
import string
import time
import random
import csv
import difflib

In [2]:
# Uncomment to display full text in cells
pd.set_option('display.max_colwidth', -1)

In [3]:
# Code from https://marcobonzanini.com/2015/01/12/searching-pubmed-with-python/
def search(query):
    Entrez.email = 'lucia.santamaria@ymail.com'
    handle = Entrez.esearch(db='pubmed', 
                            sort='relevance', 
                            retmax='20',
                            retmode='xml', 
                            term=query)
    results = Entrez.read(handle)
    return results

def fetch_details(id_list):
    ids = ','.join(id_list)
    Entrez.email = 'lucia.santamaria@ymail.com'
    handle = Entrez.efetch(db='pubmed',
                           retmode='xml',
                           id=ids)
    results = Entrez.read(handle)
    return results

In [4]:
df = pd.read_excel('Filardo_testdata.xlsx')
df.head()

,Article_ID,Full_Title,Link_to_Article,Journal,Year,Month,First Author Gender
0,Aaby et al. (2010),Non-specific effects of standard measles vaccine at 4.5 and 9 months of age on childhood mortality: randomised controlled trial,NaN,BMJ,2010,12 - Dec,Male
1,Aaron et al. (2007),Tiotropium in Combination with Placebo| Salmeterol| or Fluticasone–Salmeterol for Treatment of Chronic Obstructive Pulmonary Disease: A Randomized Trial,http://annals.org/article.aspx?articleid=734106,Annals of Internal Medicine,2007,04 - Apr,Male
2,Abbott et al. (2004),?-Blocker Use in Long-term Dialysis Patients Association With Hospitalized Heart Failure and Mortality,http://archinte.jamanetwork.com/article.aspx?articleid=217784,Archives of Internal Medicine,2004,12 - Dec,Male
3,Abdulla et al. (2008),Safety and Immunogenicity of RTS|S/AS02D Malaria Vaccine in Infants,http://www.nejm.org/doi/pdf/10.1056/NEJMoa0807773,NEJM,2008,12 - Dec,Male
4,Aboa-Eboule et al. (2007),Job Strain and Risk of Acute Recurrent Coronary Heart Disease Events,http://jama.jamanetwork.com/article.aspx?articleid=209121,JAMA,2007,10 - Oct,Female


In [5]:
def build_pubmed_query(row):
    # Extract first author, year
    # Remove final year and take everything before et al.
    author = ' '.join(row['Article_ID'].split()[:-1]).split('et al')[0].strip()
    collaboration_words = ['group', 'collaboration']
    if len(author.split()) > 3 or any(word in author.lower() for word in collaboration_words):
        # If author name has more than 3 words or it contains the coll words it's likely a collaboration - skip
        author = ''
    year = row['Year']
    # Process title -> remove punctuation and short words
    title = row['Full_Title'].translate(str.maketrans(string.punctuation, ' '*len(string.punctuation)))
    shorttitle = ' '.join([t for t in title.split() if len(t)>=5])
    return ' '.join([author, str(year), shorttitle]).strip()

In [6]:
df['pubmed_query'] = df.apply(lambda r: build_pubmed_query(r), axis=1)
df.head()

,Article_ID,Full_Title,Link_to_Article,Journal,Year,Month,First Author Gender,pubmed_query
0,Aaby et al. (2010),Non-specific effects of standard measles vaccine at 4.5 and 9 months of age on childhood mortality: randomised controlled trial,NaN,BMJ,2010,12 - Dec,Male,Aaby 2010 specific effects standard measles vaccine months childhood mortality randomised controlled trial
1,Aaron et al. (2007),Tiotropium in Combination with Placebo| Salmeterol| or Fluticasone–Salmeterol for Treatment of Chronic Obstructive Pulmonary Disease: A Randomized Trial,http://annals.org/article.aspx?articleid=734106,Annals of Internal Medicine,2007,04 - Apr,Male,Aaron 2007 Tiotropium Combination Placebo Salmeterol Fluticasone–Salmeterol Treatment Chronic Obstructive Pulmonary Disease Randomized Trial
2,Abbott et al. (2004),?-Blocker Use in Long-term Dialysis Patients Association With Hospitalized Heart Failure and Mortality,http://archinte.jamanetwork.com/article.aspx?articleid=217784,Archives of Internal Medicine,2004,12 - Dec,Male,Abbott 2004 Blocker Dialysis Patients Association Hospitalized Heart Failure Mortality
3,Abdulla et al. (2008),Safety and Immunogenicity of RTS|S/AS02D Malaria Vaccine in Infants,http://www.nejm.org/doi/pdf/10.1056/NEJMoa0807773,NEJM,2008,12 - Dec,Male,Abdulla 2008 Safety Immunogenicity AS02D Malaria Vaccine Infants
4,Aboa-Eboule et al. (2007),Job Strain and Risk of Acute Recurrent Coronary Heart Disease Events,http://jama.jamanetwork.com/article.aspx?articleid=209121,JAMA,2007,10 - Oct,Female,Aboa-Eboule 2007 Strain Acute Recurrent Coronary Heart Disease Events


In [7]:
def query_pubmed(query, title):
    results = search(query)
    if results['Count'] == '1':
        # If search returned a unique item the matching is straightforward
        paper = fetch_details(results['IdList'])
        pubmed_id = int(results['IdList'][0])
        author = paper['PubmedArticle'][0]['MedlineCitation']['Article']['AuthorList'][0]
    elif int(results['Count']) > 1 and int(results['Count']) <= 10:
        # If multiple papers (but less than 10) are returned, we compare titles and find the closest
        pubmed_ids = results['IdList']
        papers = fetch_details(pubmed_ids)
        titles = [p['MedlineCitation']['Article']['ArticleTitle'] for p in papers['PubmedArticle']]
        authors = [p['MedlineCitation']['Article']['AuthorList'] for p in papers['PubmedArticle']]
        to_match_title = title
        # Compare each of the titles with the original one
        scores = [(pid, author, difflib.SequenceMatcher(None, to_match_title.lower(), tit.lower()).ratio()) \
                  for (pid, author, tit) in zip(pubmed_ids, authors, titles)]
        # The matched article is that with the highest matching score
        matched = max(scores, key=lambda x:x[2])
        pubmed_id = int(matched[0])
        author = matched[1][0]
    else:
        return 0, None
    return pubmed_id, author

In [8]:
df.pubmed_query[0]

'Aaby 2010 specific effects standard measles vaccine months childhood mortality randomised controlled trial'

In [9]:
query_pubmed(df.pubmed_query[0], df.Full_Title[0])

(21118875,
 DictElement({'Identifier': [], 'AffiliationInfo': [{'Identifier': [], 'Affiliation': 'Bandim Health Project, Indepth Network, Bissau, Guinea-Bissau. p.aaby@bandim.org'}], 'LastName': 'Aaby', 'ForeName': 'Peter', 'Initials': 'P'}, attributes={'ValidYN': 'Y'}))

In [10]:
# Run PubMed queries sequentially
# Sleep to avoid hitting the API too often. 
# Guidelines: https://www.ncbi.nlm.nih.gov/books/NBK25497/#chapter2.Usage_Guidelines_and_Requiremen
# Store failures for further retrieval
failed = []
for i, row in df.iterrows():
    if i % 100 == 0:
        print('Queried {} entries'.format(i))
    # Sleep 0 or 1 sec each time randomly
    time.sleep(random.randint(0,1))
    try:
        pubmed_id, author = query_pubmed(row['pubmed_query'], row['Full_Title'])
        df.loc[i,'pubmed_id'], df.loc[i,'ForeName'], df.loc[i,'LastName'] = pubmed_id, author['ForeName'], author['LastName']
    except:
        failed.append(i)

Queried 0 entries
Queried 100 entries
Queried 200 entries
Queried 300 entries
Queried 400 entries
Queried 500 entries
Queried 600 entries
Queried 700 entries
Queried 800 entries
Queried 900 entries
Queried 1000 entries
Queried 1100 entries
Queried 1200 entries
Queried 1300 entries
Queried 1400 entries
Queried 1500 entries
Queried 1600 entries
Queried 1700 entries
Queried 1800 entries
Queried 1900 entries
Queried 2000 entries
Queried 2100 entries
Queried 2200 entries
Queried 2300 entries
Queried 2400 entries
Queried 2500 entries
Queried 2600 entries
Queried 2700 entries
Queried 2800 entries
Queried 2900 entries
Queried 3000 entries
Queried 3100 entries
Queried 3200 entries


In [15]:
df[df.index.isin(failed)].head()

,Article_ID,Full_Title,Link_to_Article,Journal,Year,Month,First Author Gender,pubmed_query,pubmed_id,ForeName,LastName
13,ACTIVE Writing Group (2006),Clopidogrel plus aspirin versus oral anticoagulation for atrial fibrillation in the Atrial fibrillation Clopidogrel Trial with Irbesartan for prevention of Vascular Events (ACTIVE W): a randomised controlled trial,http://www.thelancet.com/journals/lancet/article/PIIS0140-6736(06)68845-4/abstract,The Lancet,2006,06 - Jun,Unknown,2006 Clopidogrel aspirin versus anticoagulation atrial fibrillation Atrial fibrillation Clopidogrel Trial Irbesartan prevention Vascular Events ACTIVE randomised controlled trial,NaN,NaN,NaN
34,Albalak et al. (2007),Rachel Albalak| PhD; Richard J. O’Brien| MD; J. Steve Kammerer| MBA; Sean M. O’Brien| PhD; Suzanne M. Marks| MPH| MA; Kenneth G. Castro| MD; Marisa Moore| MD,http://archinte.jamanetwork.com/article.aspx?articleid=770021,Archives of Internal Medicine,2007,12 - Dec,Female,Albalak 2007 Rachel Albalak Richard O’Brien Steve Kammerer O’Brien Suzanne Marks Kenneth Castro Marisa Moore,NaN,NaN,NaN
91,Aronson et al. (1997),Use of Testosterone To Prevent Cyclophosphamide-Induced Azoospermia,http://annals.org/article.aspx?articleid=710324,Annals of Internal Medicine,1997,02 - Feb,Male,Aronson 1997 Testosterone Prevent Cyclophosphamide Induced Azoospermia,NaN,NaN,NaN
94,Arteriovenous Malformation Study Group (1999),Arteriovenous Malformations of the Brain in Adults,http://www.nejm.org/doi/full/10.1056/NEJM199906103402307,NEJM,1999,06 - Jun,Unknown,1999 Arteriovenous Malformations Brain Adults,NaN,NaN,NaN
103,Atrial Fibrillation Investigators (1998),Echocardiographic Predictors of Stroke in Patients With Atrial Fibrillation: A Prospective Study of 1066 Patients From 3 Clinical Trials,http://archinte.jamanetwork.com/article.aspx?articleid=207367,Archives of Internal Medicine,1998,06 - Jun,Male,Atrial Fibrillation Investigators 1998 Echocardiographic Predictors Stroke Patients Atrial Fibrillation Prospective Study Patients Clinical Trials,NaN,NaN,NaN


In [12]:
len(df[df.pubmed_id.isnull()])

135

In [13]:
df.to_csv("test_data_filardo.csv", 
          quoting=csv.QUOTE_ALL,
          index=False)

### Start here if the export datafile is present

In [81]:
df = pd.read_csv('test_data_filardo.csv')

In [215]:
# Remove entries that weren't matched to pubmed
print('Entries in Filardo et al: {}'.format(len(df)))
print('Entries not matched to PubMed: {}'.format(len(df[df.pubmed_id.isnull()])))
notmatched = df[df.pubmed_id.isnull()]
df = df[~df.pubmed_id.isnull()]
df.head()

Entries in Filardo et al: 3204
Entries not matched to PubMed: 145


,Article_ID,Full_Title,Link_to_Article,Journal,Year,Month,First Author Gender,pubmed_query,pubmed_id,ForeName,LastName
0,Aaby et al. (2010),Non-specific effects of standard measles vaccine at 4.5 and 9 months of age on childhood mortality: randomised controlled trial,NaN,BMJ,2010,12 - Dec,Male,Aaby 2010 specific effects standard measles vaccine months childhood mortality randomised controlled trial,21118875.0,Peter,Aaby
1,Aaron et al. (2007),Tiotropium in Combination with Placebo| Salmeterol| or Fluticasone–Salmeterol for Treatment of Chronic Obstructive Pulmonary Disease: A Randomized Trial,http://annals.org/article.aspx?articleid=734106,Annals of Internal Medicine,2007,04 - Apr,Male,Aaron 2007 Tiotropium Combination Placebo Salmeterol Fluticasone–Salmeterol Treatment Chronic Obstructive Pulmonary Disease Randomized Trial,17310045.0,Shawn D,Aaron
2,Abbott et al. (2004),?-Blocker Use in Long-term Dialysis Patients Association With Hospitalized Heart Failure and Mortality,http://archinte.jamanetwork.com/article.aspx?articleid=217784,Archives of Internal Medicine,2004,12 - Dec,Male,Abbott 2004 Blocker Dialysis Patients Association Hospitalized Heart Failure Mortality,15596637.0,Kevin C,Abbott
3,Abdulla et al. (2008),Safety and Immunogenicity of RTS|S/AS02D Malaria Vaccine in Infants,http://www.nejm.org/doi/pdf/10.1056/NEJMoa0807773,NEJM,2008,12 - Dec,Male,Abdulla 2008 Safety Immunogenicity AS02D Malaria Vaccine Infants,19064623.0,Salim,Abdulla
4,Aboa-Eboule et al. (2007),Job Strain and Risk of Acute Recurrent Coronary Heart Disease Events,http://jama.jamanetwork.com/article.aspx?articleid=209121,JAMA,2007,10 - Oct,Female,Aboa-Eboule 2007 Strain Acute Recurrent Coronary Heart Disease Events,17925517.0,Corine,Aboa-Eboulé


In [95]:
list(notmatched.pubmed_query[:10])

['Abou-Setta 2011 Comparative Effectiveness Management Interventions Fracture Systematic Review',
 'Abraham 2002 Cardiac Resynchronization Chronic Heart Failure',
 'ACTIVE 2006 Clopidogrel aspirin versus anticoagulation atrial fibrillation Atrial fibrillation Clopidogrel Trial Irbesartan prevention Vascular Events ACTIVE randomised controlled trial',
 'Adzick 2000 Pediatric Surgery',
 'Albalak 2007 Rachel Albalak Richard O’Brien Steve Kammerer O’Brien Suzanne Marks Kenneth Castro Marisa Moore',
 'Alexander 2008 National Trends Treatment Diabetes Mellitus',
 'Allen 2013 Lactobacilli bifidobacteria prevention antibiotic associated diarrhoea Clostridium difficile diarrhoea older inpatients PLACIDE randomised double blind placebo controlled multicentre trial',
 'Alvarez-Blasko 2006 Prevalence Characteristics Polycystic Ovary Syndrome Overweight Obese Women',
 'Anderson 2014 Rates Cardiopulmonary Resuscitation Training United States',
 'Anstrom 2008 Clinical Outcomes Following Coronary Sten

In [110]:
list(notmatched.Full_Title[:10])

['Comparative Effectiveness of Pain Management Interventions for Hip Fracture: A Systematic Review',
 'Cardiac Resynchronization in Chronic Heart Failure',
 'Clopidogrel plus aspirin versus oral anticoagulation for atrial fibrillation in the Atrial fibrillation Clopidogrel Trial with Irbesartan for prevention of Vascular Events (ACTIVE W): a randomised controlled trial',
 'Pediatric Surgery',
 'Rachel Albalak| PhD; Richard J. O’Brien| MD; J. Steve Kammerer| MBA; Sean M. O’Brien| PhD; Suzanne M. Marks| MPH| MA; Kenneth G. Castro| MD; Marisa Moore| MD',
 'National Trends in Treatment of Type 2 Diabetes Mellitus| 1994-2007',
 'Lactobacilli and bifidobacteria in the prevention of antibiotic-associated diarrhoea and Clostridium difficile diarrhoea in older inpatients (PLACIDE): a randomised| double-blind| placebo-controlled| multicentre trial',
 'Prevalence and Characteristics of the Polycystic Ovary Syndrome in Overweight and Obese Women',
 'Rates of Cardiopulmonary Resuscitation Training 

In [158]:
authors = [p['MedlineCitation']['Article']['AuthorList'] for p in papers['PubmedArticle']]
scores = [(pid, author, difflib.SequenceMatcher(None, to_match_title, tit).ratio()) \
          for (pid, author, tit) in zip(pubmed_ids, authors, titles)]
matched = max(scores, key=lambda x:x[2])

In [191]:
s = 'Abraham 2002 Cardiac Resynchronization Chronic Heart Failure'
query_pubmed(s)

(12063368,
 DictElement({'Identifier': [], 'AffiliationInfo': [{'Identifier': [], 'Affiliation': 'Division of Cardiovascular Medicine, University of Kentucky College of Medicine, Lexington 40536-0284, USA. wtabra2@uky.edu'}], 'LastName': 'Abraham', 'ForeName': 'William T', 'Initials': 'WT'}, attributes={'ValidYN': 'Y'}))

In [166]:
author[0]['ForeName']

'William T'

In [168]:
[au[0]['ForeName'] for au in authors]

['William T', 'Finlay A', 'William T']

In [169]:
[au[0]['LastName'] for au in authors]

['Abraham', 'McAlister', 'Abraham']

In [170]:
au2 = query_pubmed(df.pubmed_query[0])
au2

(21118875,
 DictElement({'Identifier': [], 'AffiliationInfo': [{'Identifier': [], 'Affiliation': 'Bandim Health Project, Indepth Network, Bissau, Guinea-Bissau. p.aaby@bandim.org'}], 'LastName': 'Aaby', 'ForeName': 'Peter', 'Initials': 'P'}, attributes={'ValidYN': 'Y'}))

In [176]:
type(au2[1])

Bio.Entrez.Parser.StructureElement

In [181]:
type(author)

Bio.Entrez.Parser.ListElement

In [183]:
isinstance(author, Entrez.Parser.ListElement)

True

In [120]:
' '.join(('Abraham 2002 Cardiac Resynchronization Chronic Heart Failure').split())[2:]

'raham 2002 Cardiac Resynchronization Chronic Heart Failure'

In [128]:
ss = 'Abraham 2002 Cardiac Resynchronization Chronic Heart Failure'
to_match_title = ' '.join(ss.split()[2:])

In [96]:
query_pubmed('Abraham 2002 Cardiac Resynchronization Chronic Heart Failure')

(None, None)

In [98]:
res = search('Abraham 2002 Cardiac Resynchronization Chronic Heart Failure')

In [114]:
pubmed_ids = res['IdList']
pubmed_ids

['12063368', '15353430', '12151867']

In [102]:
papers = fetch_details(res['IdList'])

In [115]:
titles = [p['MedlineCitation']['Article']['ArticleTitle'] for p in papers['PubmedArticle']]
titles

['Cardiac resynchronization in chronic heart failure.',
 'Systematic review: cardiac resynchronization in patients with symptomatic heart failure.',
 'Cardiac resynchronization therapy for heart failure: biventricular pacing and beyond.']

In [145]:
scores = [(pid, difflib.SequenceMatcher(None, to_match_title, tit).ratio()) for (pid, tit) in zip(pubmed_ids, titles)]
max(scores, key=lambda x:x[1])[0]

'12063368'

In [112]:
import difflib
difflib.SequenceMatcher(None, 'hello world', 'hello').ratio()

0.625

In [113]:
[difflib.SequenceMatcher(None, s, p['MedlineCitation']['Article']['ArticleTitle']).ratio() for p in papers['PubmedArticle']]

[0.9108910891089109, 0.6231884057971014, 0.562962962962963]

In [65]:
def parse_forename(forename):
    name = str(forename).lower()
    if len(name.split()) > 1:
        first_name, middle_name = name.split()[:2]
    else:
        first_name, middle_name = name, ''
    return first_name, middle_name

In [72]:
df['first_name'], df['middle_name'] = zip(*df['ForeName'].map(parse_forename))
df['last_name'] = df['LastName'].apply(lambda n: str(n).lower())

In [73]:
df.head()

,Article_ID,Full_Title,Link_to_Article,Journal,Year,Month,First Author Gender,pubmed_query,pubmed_id,ForeName,LastName,first_name,middle_name,last_name
0,Aaby et al. (2010),Non-specific effects of standard measles vacci...,NaN,BMJ,2010,12 - Dec,Male,Aaby 2010 specific effects standard measles va...,21118875.0,Peter,Aaby,peter,,aaby
1,Aaron et al. (2007),Tiotropium in Combination with Placebo| Salmet...,http://annals.org/article.aspx?articleid=734106,Annals of Internal Medicine,2007,04 - Apr,Male,Aaron 2007 Tiotropium Combination Placebo Salm...,17310045.0,Shawn D,Aaron,shawn,d,aaron
2,Abbott et al. (2004),?-Blocker Use in Long-term Dialysis Patients A...,http://archinte.jamanetwork.com/article.aspx?a...,Archives of Internal Medicine,2004,12 - Dec,Male,Abbott 2004 Blocker Dialysis Patients Associat...,15596637.0,Kevin C,Abbott,kevin,c,abbott
3,Abdulla et al. (2008),Safety and Immunogenicity of RTS|S/AS02D Malar...,http://www.nejm.org/doi/pdf/10.1056/NEJMoa0807773,NEJM,2008,12 - Dec,Male,Abdulla 2008 Safety Immunogenicity AS02D Malar...,19064623.0,Salim,Abdulla,salim,,abdulla
4,Aboa-Eboule et al. (2007),Job Strain and Risk of Acute Recurrent Coronar...,http://jama.jamanetwork.com/article.aspx?artic...,JAMA,2007,10 - Oct,Female,Aboa-Eboule 2007 Strain Acute Recurrent Corona...,17925517.0,Corine,Aboa-Eboulé,corine,,aboa-eboulé


In [76]:
len(df)

3204

In [75]:
len(df[df.pubmed_id.isnull()])

471

In [77]:
df['First Author Gender'].unique()

array(['Male', 'Female', 'Unknown'], dtype=object)

In [78]:
gender_keys = {'Unknown': 'u', 
               'Male': 'm',
               'Female': 'f'}

In [82]:
def clean_names(df, data, column):
    """keep the string in column 'column' if it has more than one character, otherwise replace by ''. """
    def try_to_simplify(s):
        try:
            if len(s)>1:
                return s
            else:
                return ''
        except:
            return ''
    df[column] = data[column].map(lambda x: try_to_simplify(x))

In [87]:
test_data = pd.DataFrame()
test_data['first_name'], test_data['middle_name'], test_data['last_name'] = df['first_name'], df['middle_name'], df['last_name']
test_data['gender'] = df['First Author Gender'].apply(lambda n: gender_keys[n])
#clean_names(test_data, df, 'first_name')
#clean_names(test_data, df, 'middle_name')
#test_data['full_name'] = df.apply(lambda x: x.ForeName + ' ' + x.LastName)

In [88]:
test_data

,first_name,middle_name,last_name,gender
0,peter,,aaby,m
1,shawn,d,aaron,m
2,kevin,c,abbott,m
3,salim,,abdulla,m
4,corine,,aboa-eboulé,f
5,nan,,nan,m
6,nan,,nan,m
7,bo,,abrahamsen,m
8,donald,i,abrams,m
9,nancy,j,aburto,f
